In [20]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [21]:
train = pd.read_csv('train.csv', encoding='big5')
test = pd.read_csv('test.csv', encoding='big5')
train = train[train['測項'] == 'PM2.5']
train

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
9,2014/1/1,豐原,PM2.5,26,39,36,35,31,28,25,...,36,45,42,49,45,44,41,30,24,13
27,2014/1/2,豐原,PM2.5,21,23,30,30,22,18,13,...,53,43,43,45,46,32,16,19,22,26
45,2014/1/3,豐原,PM2.5,19,25,27,20,16,14,15,...,32,36,34,45,40,41,23,29,23,37
63,2014/1/4,豐原,PM2.5,27,27,14,20,22,24,26,...,62,55,56,67,78,83,90,75,85,82
81,2014/1/5,豐原,PM2.5,80,80,76,81,75,66,70,...,64,73,57,57,53,70,70,60,68,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4239,2014/12/16,豐原,PM2.5,5,9,17,26,33,32,24,...,8,16,28,37,39,36,36,31,26,19
4257,2014/12/17,豐原,PM2.5,21,26,26,31,29,21,13,...,12,6,4,11,11,5,0,6,6,5
4275,2014/12/18,豐原,PM2.5,0,2,2,3,3,3,11,...,31,31,36,31,31,34,39,36,24,23
4293,2014/12/19,豐原,PM2.5,14,16,13,14,21,19,23,...,21,21,19,19,25,19,11,0,7,7


In [22]:
test = test[test['AMB_TEMP'] == 'PM2.5']

In [23]:
test

,id_0,AMB_TEMP,21,21.1,20,20.1,19,19.1,19.2,18,17
8,id_0,PM2.5,33,39,39,25,18,18,17,9,4
26,id_1,PM2.5,17,22,17,9,17,23,29,17,18
44,id_2,PM2.5,50,70,70,73,61,55,47,38,26
62,id_3,PM2.5,85,65,47,30,20,13,7,4,4
80,id_4,PM2.5,17,8,14,8,8,11,23,32,30
...,...,...,...,...,...,...,...,...,...,...,...
4238,id_235,PM2.5,63,58,69,66,68,59,64,59,51
4256,id_236,PM2.5,49,66,93,106,111,96,84,79,75
4274,id_237,PM2.5,59,43,43,38,48,54,42,46,39
4292,id_238,PM2.5,12,8,7,11,11,3,0,0,0


In [24]:
train = train.drop(['日期', '測站', '測項'], axis=1)
test_x = test.iloc[:, 2:]

In [25]:
train_x = []
train_y = []

In [26]:
for i in range(15):
    x = train.iloc[:, i : i + 9]
    x.columns = np.array(range(9))
    y = train.iloc[:, i + 9]
    y.columns = np.array(range(1))
    train_x.append(x)
    train_y.append(y)

In [27]:
# train_x

In [28]:
train_x = pd.concat(train_x)
train_y = pd.concat(train_y)

In [29]:
# train_x

In [30]:
train_y = np.array(train_y, float)
test_x = np.array(test_x, float)

In [31]:
ss = StandardScaler()
ss.fit(train_x)
train_x = ss.transform(train_x)

ss.fit(test_x)
test_x = ss.transform(test_x)

In [32]:
def r2_score(y_true, y_predict):
    MSE = np.sum((y_true - y_predict) ** 2) / len(y_true)
    return 1 - MSE / np.var(y_true)

In [33]:
class LinearRegression:
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None
        self._theta = None
    def fit_normal(self, X_train, y_train):
        assert X_train.shape[0] == y_train.shape[0]
        X_b = np.hstack([np.ones((len(X_train), 1)), X_train])
        self._theta = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y_train)
        
        self.intercept_ = self._theta[0]
        self.coef_ = self._theta[1:]
        
        return self
    def fit_gd(self, X_train, y_train, eta=0.01, n_iters=1e4):
        assert X_train.shape[0] == y_train.shape[0]
        def Loss(theta, X_b, y):
            try:
                return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
            except:
                return float('inf')
        def dLoss(theta, X_b, y):
            return X_b.T.dot(X_b.dot(theta) - y) * 2. / len(y)
        def gradient_descent(X_b, y, initial_theta, eta, n_iters=1e4, eps=1e-8):
            theta = initial_theta
            cur_iter = 0
            while cur_iter < n_iters:
                gradient = dLoss(theta, X_b, y)
                last_theta = theta
                theta = theta - eta * gradient
                if(abs(Loss(theta, X_b, y) - Loss(last_theta, X_b, y)) < eps):
                    break
                cur_iter += 1
            return theta
        X_b = np.hstack([np.ones((len(X_train), 1)), X_train])
        initial_theta = np.zeros(X_b.shape[1])
        self._theta = gradient_descent(X_b, y_train, initial_theta, eta, n_iters)
        self.intercept_ = self._theta[0]
        self.coef_ = self._theta[1:]
        return self
    def predict(self, X_predict):
        assert self.intercept_ is not None and self.coef_ is not None
        assert X_predict.shape[1] == len(self.coef_)
        X_b = np.hstack([np.ones((len(X_predict), 1)), X_predict])
        return X_b.dot(self._theta)
    def score(self, X_test, y_test):
        y_predict = self.predict(X_test)
        return r2_score(y_test, y_predict)
    def __repr__(self):
        return

In [34]:
LR = LinearRegression().fit_gd(train_x, train_y)

In [35]:
LR.score(train_x, train_y)

0.854900184173953

In [36]:
result = LR.predict(test_x)

In [38]:
sampleSubmission = pd.read_csv('sample_submission.csv', engine='python', encoding='gbk')
sampleSubmission['value'] = result
sampleSubmission.to_csv('result.csv')